In [1]:
import random
import hashlib
import json
from faker import Faker
fake = Faker('ru_RU')

In [ ]:
def gen_group(n=5):
    group = ('ИУ','АК','МТ','РК','ФМ')
    return random.choice(group) + str(random.randint(1, n))

In [ ]:
def gen_teacher_position():
    positions = ('профессор','доцент','старший преподаватель','преподаватель','ассистент','заведующий кафедрой')
    return random.choices(positions, (8, 20, 20, 20, 10, 1), k=1)[0]

In [ ]:
def gen_course_name(n=1):
    with open('subject.json','r') as f:
        subjects = json.load(f)
    return random.choice(subjects) + ' ' + str(random.randint(1, n))

In [ ]:
def gen_name_gender():
    gender = random.choice(('M', 'F'))
    name = fake.name_male() if gender == 'M' else fake.name_female()
    name = name.split()
    if len(name) > 3: name = name[1:4]
    return name + [gender]

In [ ]:
def gen_birthday_role(role):
    if role == 'student':
        birthday = fake.date_of_birth(minimum_age=18, maximum_age=30)
    else:
        birthday = fake.date_of_birth(minimum_age=30, maximum_age=70)
    return [birthday.isoformat(), role]

In [ ]:
def gen_phone_number():
    num = fake.phone_number()
    num = num.replace(' ','').replace('-','').replace('+7','8').replace('(','').replace(')','')
    return num

In [ ]:
def gen_salt_hash():
    password = fake.password()
    salt = fake.password(length=8)
    return [salt, hashlib.sha1((password + salt).encode('utf-8')).hexdigest()]

In [ ]:
class Generator:

    def __init__(self, folder):
        self.folder = folder
        self.student_id = 0
        self.teacher_id = 0
        self.account_id = 0
        self.course_id = 0

        with open(folder + 'account.csv', 'w') as f:
            f.write('Account ID,First name,Last name,Mid name,Gender,Birthday,Role,Email,Phone number,Salt,Hash\n')
        with open(folder + 'student.csv', 'w') as f:
            f.write('Student ID,Group,Account ID\n')
        with open(folder + 'teacher.csv', 'w') as f:
            f.write('Teacher ID,Position,Account ID\n')
        with open(folder + 'course.csv', 'w') as f:
            f.write('Course ID,Name,Teacher ID\n')
        with open(folder + 'lesson.csv', 'w') as f:
            f.write('Day,Time,Place,Type,Course ID\n')
        with open(folder + 'student_course.csv', 'w') as f:
            f.write('Student ID,Course ID,Score\n')

    def gen_student(self, n):
        with open(self.folder + 'student.csv', 'a') as f:
            for i in range(n):
                self.student_id += 1
                self.account_id += 1
                f.write(','.join((str(self.student_id), gen_group(), str(self.account_id))))
                f.write('\n')
                self.gen_account()
                
    def gen_teacher(self, n):
        with open(self.folder + 'teacher.csv', 'a') as f:
            for i in range(n):
                self.teacher_id += 1
                self.account_id += 1
                f.write(','.join((str(self.teacher_id), gen_teacher_position(), str(self.account_id))))
                f.write('\n')
                self.gen_account('teacher')

    def gen_account(self, role='student'):
        with open(self.folder + 'account.csv', 'a') as f:
            account = [str(self.account_id)]
            account += gen_name_gender()
            account += gen_birthday_role(role)
            account.append(fake.email())
            account.append(gen_phone_number())
            account += gen_salt_hash()
            f.write(','.join(account))
            f.write('\n')
            
    def gen_course(self, n, n_course):
        with open(self.folder + 'course.csv', 'a') as f:
            for i in range(n):
                self.course_id += 1
                f.write(','.join((str(self.course_id), gen_course_name(n_course), str(random.randint(1, self.teacher_id)))))
                f.write('\n')
                
    def gen_lesson(self, n):
        days = ('Пн','Вт','Ср','Чт','Пт','Сб','Вс')
        with open(self.folder + 'lesson.csv', 'a') as f:
            for i in range(n):
                f.write('{},{},{}{:02}{:02},{},{}\n'.format(random.choice(days), random.randint(1, 10),
                                                 random.randint(1, 30), random.randint(1, 20), random.randint(1, 90),
                                                 random.choice(('сем','лек','лаб')),
                                                   random.randint(1, self.course_id)))

    def gen_student_course(self, n):
        with open(self.folder + 'student_course.csv', 'a') as f:
            for i in range(n):
                f.write('{},{},{}\n'.format(random.randint(1, self.student_id), random.randint(1, self.course_id), random.randint(1, 100)))
                

    def gen_all(self, n_teacher, n_student):
        self.gen_teacher(n_teacher)
        self.gen_student(n_student)
        self.gen_course(self.teacher_id*2, max(1, self.teacher_id/100))
        self.gen_lesson(self.teacher_id*4)
        self.gen_student_course(self.student_id*4)

In [ ]:
gen = Generator('csv/')
gen.gen_all(1000, 20000)

In [ ]:
with open('csv/lesson.csv', 'w') as f:
            f.write('Day,Time,Place,Type,Course ID\n')
gen.gen_lesson(4000)